1. Indexer
    - Crawler (Focused) => Repository(Collection)
    - Document Analyzer => HTML, Tokenizing,Normalizing
                           Stemming(BPE), N-gram, MA, POS
                           Stopwords, RE, Phrases
                        => Preprocessing
    - Features => Lexicon(사전)
    - Document (Query) Representation => BoW
      Document-Term Mat.(DTM), Term-Document Mat.(TDM)(핵심)
      => Inverted Document Indexing(역문헌구조)
2. Relevance(Ranking)
    - Weighting(TF-IDF), Similarity(Cosine:0-1,Euclidean)
    - Sortting
3. Results
    - Top K

In [6]:
# Lexicon
from konlpy.corpus import kobill

def getLexicon1() :
    lexicon = list()
    for document in [ kobill.open(idx).read() for idx in kobill.fileids() ] :
        for term in document.split() :
            if term not in lexicon :
                lexicon.append(term)
    return lexicon

def getLexicon2() :
    lexicon = list()
    for document in [ kobill.open(idx).read() for idx in kobill.fileids() ] :
        for term in document.split() :
            lexicon.append(term)
    return list(set(lexicon))

In [7]:
%timeit getLexicon1()

71.3 ms ± 441 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit getLexicon2()

2.94 ms ± 17.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [9]:
lexicon = getLexicon2()

In [15]:
len(lexicon)
#            1 2 3 .... 2638  => Bag of Words
# doc1       1 0 1  ... 
# doc2       1 1 1  ...
# ... 
# doc10      0 0 1  ...

2638

In [56]:
from collections import defaultdict

def documentRepresentation1() :
    documentList = list()
    for document in [kobill.open(idx).read() for idx in kobill.fileids()] :
        bow = list(0 for _ in range(len(lexicon)))
        for term in document.split():
            bow[lexicon.index(term)] = 1
        documentList.append(bow)
    return documentList

def documentRepresentation2() :
    documentList = list()
    for document in [kobill.open(idx).read() for idx in kobill.fileids()] :
        bow = dict()
        for term in document.split():
            bow[lexicon.index(term)] = 1
        documentList.append(bow)
    return documentList

def documentRepresentation3() :
    documentList = list()
    for document in [kobill.open(idx).read() for idx in kobill.fileids()] :
        bow = dict()
        for term in document.split():
            bow[term] = 1
        documentList.append(bow)
    return documentList

def documentRepresentation4() :
    documentList = list()
    for document in [kobill.open(idx).read() for idx in kobill.fileids()] :
        bow = dict()
        for term in document.split():
            if term in bow.keys() :
                bow[term] += 1
            else :
                bow[term] = 1
        documentList.append(bow)
    return documentList


def documentRepresentation5() :
    documentList = list()
    for document in [kobill.open(idx).read() for idx in kobill.fileids()] :
        bow = defaultdict(int)
        for term in document.split():
            bow[term] += 1
        documentList.append(bow)
    return documentList

def documentRepresentation6() :
    documentList = defaultdict(lambda:defaultdict(int))
    for idx in kobill.fileids() :
        for term in kobill.open(idx).read().split():
            ## split : Preprocessing 처리
            documentList[idx][term] += 1
            #documentList.keys() => collection
            #set(documentList.values().keys()) => lexicon
    return documentList 


In [57]:
temp = documentRepresentation6()
#len(temp[0].keys()) # => 0:없음, 1:있음
temp[0]

defaultdict(int, {})

In [41]:
%timeit documentRepresentation1()

160 ms ± 3.1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [34]:
%timeit documentRepresentation2()

160 ms ± 1.83 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [35]:
%timeit documentRepresentation3()

2.76 ms ± 27.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [42]:
%timeit documentRepresentation4()

3.55 ms ± 48.5 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [58]:
%timeit documentRepresentation5()

3.55 ms ± 62 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [59]:
%timeit documentRepresentation6()

3.73 ms ± 46.8 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [61]:
docList = documentRepresentation6()
docList.keys()

dict_keys(['1809890.txt', '1809891.txt', '1809892.txt', '1809893.txt', '1809894.txt', '1809895.txt', '1809896.txt', '1809897.txt', '1809898.txt', '1809899.txt'])

In [64]:
docList['1809890.txt']

defaultdict(int,
            {'지방공무원법': 4,
             '일부개정법률안': 3,
             '(정의화의원': 1,
             '대표발의': 1,
             ')': 2,
             '의': 4,
             '안': 2,
             '번': 1,
             '호': 2,
             '9890': 1,
             '발의연월일': 1,
             ':': 7,
             '2010.': 1,
             '11.': 1,
             '12.': 1,
             '발': 1,
             '자': 1,
             '정의화․이명수․김을동': 1,
             '이사철․여상규․안규백': 1,
             '황영철․박영아․김정훈': 1,
             '김학송': 1,
             '의원(10인)': 1,
             '제안이유': 1,
             '및': 4,
             '주요내용': 1,
             '초등학교': 6,
             '저학년의': 1,
             '경우에도': 2,
             '부모의': 1,
             '따뜻한': 1,
             '사랑과': 1,
             '보살핌이': 1,
             '필요': 1,
             '한': 1,
             '나이이나,': 1,
             '현재': 3,
             '공무원이': 2,
             '자녀를': 4,
             '양육하기': 2,
             '위하여': 2,
             '육아휴직을': 3,
      

In [78]:
# DTM ==> TDM
# Boolean 검색 => 집합론
query = "국회 의원 국민"
result = list()
for term in query.split() :
    searchResult = list()
    for idx, termList in docList.items() :
        if term in termList.keys() :
            #idx -> 저장
            searchResult.append(idx)
    result.append(searchResult)
    
result
# 국회:[]
# 의원:[]
# 국민:[]

[['1809897.txt', '1809898.txt'],
 ['1809896.txt'],
 ['1809896.txt', '1809897.txt']]

In [76]:
one = result.pop()
while result :
    temp = result.pop()
    one = list(set(one).intersection(temp))
    
one

[]

In [83]:
# DTM
query = "국회 의원 국민"

def booleanResult1() :
    result = list()
    for term in query.split() :
        searchResult = list()
        for idx, termList in docList.items() :
            if term in termList.keys() :
                #idx -> 저장
                searchResult.append(idx)
        result.append(searchResult)
   
    one = result.pop()
    while result :
        temp = result.pop()
        one = list(set(one).intersection(temp))
    return one

# TDM
TDM = defaultdict(lambda:defaultdict(int))
for idx, termList in docList.items() :
    for term, freq in termList.items() :
        TDM[term][idx] = freq
            
def booleanResult2() :

    result = list()
    for term in query.split():
        result.append(list(TDM[term].keys()))
        
    one = result.pop()
    while result :
        temp = result.pop()
        one = list(set(one).intersection(temp))
    return one

In [84]:
%timeit booleanResult1()

4.49 µs ± 48.6 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [85]:
%timeit booleanResult2()

2.21 µs ± 28.4 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
